# Naming convention and contents of the created files

1. `faket/class_mask.mrc` = `class_mask.mrc` sliced to valid region.
1. `faket/occupancy_mask.mrc` = `occupancy_mask.mrc` slice to valid region.
1. `reconstruction.mrc` = `faket/reconstruction_shrec.mrc` slice to valid region for tomogram 9.
1. `faket/projections_noiseless.mrc` = `grandmodel_unbinned.mrc` measured with Radon transform.
1. `faket/projections_content.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.1) shifted & scaled according its style*.
1. `faket/projections_noisy.mrc` = `faket/projections_noiseless.mrc` + noise (std=0.4) shifted & scaled according its style*.
1. `faket/projections_styled.mrc` = result of NST initialized with `faket/projections_noisy.mrc`, content `faket/projections_content.mrc`, and using its style*.
1. `faket/reconstruction_content.mrc` = reconstruction of `faket/projections_content.mrc`.
1. `faket/reconstruction_noisy.mrc` = reconstruction of `faket/projections_noisy.mrc`.
1. `faket/reconstruction_styled.mrc` = reconstruction of `faket/projections_styled.mrc`.
1. `faket/reconstruction_baseline.mrc` = reconstruction of `projections.mrc`.

\* Each time we mention style in the text above, it refers to a `projections.mrc` file from a model_N+1. In case N=8, the style is taken from N=0.

In [ ]:
%load_ext autoreload
%autoreload 2

from tqdm.notebook import tqdm

import os
import multiprocessing
import gpuMultiprocessing
from src.data import load_mrc, save_mrc
from src.data import slice_to_valid, vol_to_valid
from src.data import downsample_sinogram_space
from src.data import get_clim, get_theta
from src.data import match_mean_std, normalize
from src.transform import radon_3d, reconstruct
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data_folder = 'data/shrec2021_extended_dataset/'

In [ ]:
# SHREC21 provides the data in square shape even
# thought the data is stored only in the center
# The following values specify where to slice
z_valid = (0.32226, 0.67382)  # Valid range normalized

In [ ]:
# slice class_mask.mrc to faket/class_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, N, 'class_mask', z_valid, 
                 out_fname='faket/class_mask.mrc')

In [ ]:
# slice occupancy_mask.mrc to faket/occupancy_mask.mrc
for N in range(10):
    vol_to_valid(data_folder, N, 'occupancy_mask', z_valid, 
                 out_fname='faket/occupancy_mask.mrc')

In [ ]:
# slice reconstruction.mrc to faket/reconstruction_shrec.mrc
vol_to_valid(data_folder, 9, 'reconstruction', z_valid, 
                 out_fname='faket/reconstruction_shrec.mrc')

## Creating projections

In [ ]:
# create faket/projections_noiseless.mrc by measuring the grandmodel_unbinned.mrc with Radon transform
dose = 0
for N in range(0, 9): # We do not need this modality for the test model_9
    print(f'Processing N: {N}')
    volume = load_mrc(data_folder, N, 'grandmodel_unbinned.mrc')
    theta = get_theta(data_folder, N)
    # Circle = False because we measure with the data outside the circle 
    # but later we cut the measurements to desired shape (SHREC did it this way - confirmed from a call)
    sinogram = radon_3d(volume, theta, dose=dose, out_shape=1024, slice_axis=1, circle=False)
    save_mrc(sinogram.astype(np.float32), data_folder, N, 
             'faket/projections_noiseless.mrc', overwrite=True)

In [ ]:
# create faket/projections_content.mrc and faket/projections_noisy.mrc
for N in range(0, 9): # We do not need this modality for the test model_9
    print(f'Processing N: {N}')
    volume = load_mrc(data_folder, N, 'faket/projections_noiseless.mrc')
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    style = load_mrc(data_folder, style_N, 'projections.mrc')
    
    rng = np.random.default_rng(seed=N)
    noise = rng.normal(loc=0.0, scale=0.4, size=volume.size).reshape(volume.shape)
    
    volume  = match_mean_std(volume, style)  # Scaling per tilt (bigger the abs(angle), longer the trajectory)
    volume = normalize(volume)  # Scale between [0, 1]
    
    volume_noisy = volume + noise
    volume_noisy = np.clip(volume_noisy, *get_clim(volume_noisy, 0.0001, 0.9999))  # Remove outliers
    volume_noisy = match_mean_std(volume_noisy, style)  # Scale back to match style
    
    save_mrc(volume_noisy.astype(np.float32), data_folder, N, 
             'faket/projections_noisy.mrc', overwrite=True)
    
    volume_content = volume + noise / 4  # Same noise just 1/4 of the std
    volume_content = np.clip(volume_content, *get_clim(volume_content, 0.0001, 0.9999))  # Remove outliers
    volume_content = match_mean_std(volume_content, style)  # Scale back to match style
    
    save_mrc(volume_content.astype(np.float32), data_folder, N, 
             'faket/projections_content.mrc', overwrite=True)

### Neural Style Transfer

In [ ]:
nstc = {  # NEURAL STYLE TRANSFER BASE CONFIG
    # 'content': 'example.mrc',
    # 'style': 'example.mrc',
    # '--init': 'example.mrc',
    # '--output': 'example.mrc', 
    # '--random-seed': None,
    '--style-weights': 1.0,
    '--content-weight': 1.0, 
    '--tv-weight': 0,
    '--min-scale': 1024,
    '--end-scale': 1024,
    '--iterations': 1,
    '--initial-iterations': 1,
    '--save-every': 2,
    '--step-size': 0.15,
    '--avg-decay': 0.99,
    '--style-scale-fac': 1.0,
    '--pooling': 'max',
    '--devices': 'cuda:0',
    '--seq_start' : 0,
    '--seq_end' : 61,
}

def get_command(expname, nst_command, config):
    command = (
    f"EXPNAME={expname} {nst_command} "
    f"{config['content']} {config['style']} "
    f"{' '.join([f'{k} {v}' for k, v in config.items() if k.startswith('--')])}")
    return command

In [ ]:
# create faket/projections_styled.mrc
gpu_id_list = [6]
NST_command = 'python3 style_transfer/cli.py'

command_queue = []
for N in range(0, 9): # We do not need this modality for the test model_9
    style_N = (N + 1) % 9 # For the last train model we take style stats from the first train model
    
    EXPNAME = f'TOMOGRAM_{N}'  # Just for visualizing the progress
    tomo_folder = os.path.join(data_folder, f'model_{N}', 'faket')

    conf = nstc.copy()
    conf.update({
        'content': os.path.join(tomo_folder, 'projections_content.mrc'),
        'style': os.path.join(data_folder, f'model_{style_N}', 'projections.mrc'), 
        '--init': os.path.join(tomo_folder, 'projections_noisy.mrc'),
        '--output': os.path.join(tomo_folder, 'projections_styled.mrc'), 
        '--random-seed': N,
    })
    
    command = get_command(EXPNAME, NST_command, conf)
    command_queue.append(command)
    
# Run all the commands (returns list of failed commands if any)
gpuMultiprocessing.queue_runner(command_queue, gpu_id_list,
                                env_gpu_name='CUDA_VISIBLE_DEVICES',
                                processes_per_gpu=6, allowed_restarts=1)

## Computing reconstructions

In [ ]:
recc = {  # RECONSTRUCTION BASE CONFIG
    'downsample_angle' : 1,  # Sinogram downsampling in theta dimension (1 = no downsampling)
    'downsample_pre' : 2,  # Sinogram downsampling (1 = no downsampling)
    'order' : 3,  # Downsampling in space with spline interpolation of order (0 - 5)
    'filter' : 'ramp2d',  # Filter userd during reconstruction in FBP algorithm
    'filterkwargs' : {'crowtherFreq': 25, 'radiusCutoff': 230, 'angularCutoff': (0, 83)},
    'downsample_post' : 1,  # Reconstruction downsampling
    'ncpus': 61, # multiprocessing.cpu_count(),  # Number of CPUs to use while reconstructing
    'z_valid': z_valid # 2-tuple range of valid pixels in Z dimension normalized from 0 to 1. (0., 1.) or None for all.
}

In [ ]:
# reconstruct faket/projections_content.mrc to produce faket/reconstruction_content.mrc
for N in range(0, 9):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : 'faket/projections_content.mrc', 
        'output_mrc' : 'faket/reconstruction_content.mrc'
    })
    reconstruct(data_folder, N, conf)

In [ ]:
# reconstruct faket/projections_noisy.mrc to produce faket/reconstruction_noisy.mrc
for N in range(0, 9):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : 'faket/projections_noisy.mrc', 
        'output_mrc' : 'faket/reconstruction_noisy.mrc'
    })
    reconstruct(data_folder, N, conf)

In [ ]:
# reconstruct faket/projections_styled.mrc to produce faket/reconstruction_styled.mrc
for N in range(0, 9):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : 'faket/projections_styled.mrc', 
        'output_mrc' : 'faket/reconstruction_styled.mrc'
    })
    reconstruct(data_folder, N, conf)

In [ ]:
# reconstruct projections.mrc to produce faket/reconstruction_baseline.mrc
for N in range(0, 10):
    print(f'Processing N: {N}')
    conf = recc.copy()
    conf.update({
        'input_mrc' : 'projections.mrc', 
        'output_mrc' : 'faket/reconstruction_baseline.mrc'
    })
    reconstruct(data_folder, N, conf)

# Deep Finder experiments

In [ ]:
## SEPARATE (for sure save the model after 15 and 30 epochs)
# 1. DF('faket/reconstruction_baseline.mrc')  30 epochs on 9 tomograms
# 2. DF('faket/reconstruction_content.mrc')   30 epochs on 9 tomograms
# 3. DF('faket/reconstruction_noisy.mrc')     30 epochs on 9 tomograms
# 4. DF('faket/reconstruction_styled.mrc')    30 epochs on 9 tomograms

## FINETUNING (pre-train on synthetic & finetune on "real")
# 5. take model from step 2. saved at 15 epochs and finetune on 'faket/reconstruction_baseline.mrc' for 15 epochs
# 6. take model from step 3. saved at 15 epochs and finetune on 'faket/reconstruction_baseline.mrc' for 15 epochs
# 7. take model from step 4. saved at 15 epochs and finetune on 'faket/reconstruction_baseline.mrc' for 15 epochs

## AUGMENTATION
# 8. DF('faket/reconstruction_content.mrc'[0, 1, 2, 3, 4] & 'faket/reconstruction_baseline.mrc'[5, 6, 7, 8]) for 30 ep.
# 9. DF('faket/reconstruction_noisy.mrc'[0, 1, 2, 3, 4] & 'faket/reconstruction_baseline.mrc'[5, 6, 7, 8]) for 30 ep.
# 10. DF('faket/reconstruction_styled.mrc'[0, 1, 2, 3, 4] & 'faket/reconstruction_baseline.mrc'[5, 6, 7, 8]) for 30 ep.

# TEST on 'faket/reconstruction_baseline.mrc'
# TEST on 'faket/reconstruction_shrec.mrc'

In [ ]:
train_script="""
for idx in 1 2 3 4 5 6 7; 
do 
    PYTHONHASHSEED=0 python deepfinder/launch_training.py --path_config results/experiment_${idx}a/
done

for idx in 1 5 6 7; 
do 
    PYTHONHASHSEED=0 python deepfinder/launch_training.py --path_config results/experiment_${idx}b/
done
"""
with open('train_script.sh', 'w') as file:
  file.write(train_script)

!bash train_script.sh

In [ ]:
# run segmentation step on GPU with ID GPU_no
!python deepfinder/step1_launch_segment_loop.py --path_config results/

In [ ]:
# run clustering 
!python deepfinder/step2_clustering_loop.py --path_config results/

In [ ]:
!python deepfinder/step3_launch_evaluation.py --path_config results/